In [1]:
import pandas as pd
import requests
from wordcloud import WordCloud
import os
import seaborn as sns
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

plt.rc("font", family="Malgun Gothic")
#plt.rc("font", family="AppleGothic")
plt.rc("axes", unicode_minus=False)  #-폰트 깨지는 문제 해결

In [2]:
df = pd.read_csv('./data/전국1인가구비율.csv', encoding='cp949')
df

,행정구역별,2019,2019.1,2019.2
0,행정구역별,1인가구비율<br>(A÷B×100) (%),1인가구<br>(A) (가구),일반가구<br>(B) (가구)
1,전국,30.2,6147516,20343188
2,서울특별시,33.4,1299787,3896389
3,부산광역시,30.7,423227,1377030
4,대구광역시,29.4,284416,968620
5,인천광역시,26.6,297865,1120576
6,광주광역시,31.1,182364,587159
7,대전광역시,33.7,205165,609043
8,울산광역시,26.5,116015,437094
9,세종특별자치시,30.1,39078,129664


In [12]:
df1 = df.iloc[2:,[0,1,2]]
df1

,행정구역별,2019,2019.1
2,서울특별시,33.4,1299787
3,부산광역시,30.7,423227
4,대구광역시,29.4,284416
5,인천광역시,26.6,297865
6,광주광역시,31.1,182364
7,대전광역시,33.7,205165
8,울산광역시,26.5,116015
9,세종특별자치시,30.1,39078
10,경기도,26.3,1290893
11,강원도,32.9,208857


In [2]:
df_seoul = pd.read_csv('./data/서울시_1인가구.txt', sep = '\t', thousands = ',', engine='python', encoding='utf-8')
df_seoul


# 서울시_1인가구_거처별.txt   거처별

# 서울시_1인가구.txt         성별 

,기간,구분,성별,합계,20세미만,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85세이상
0,2019,합계,계,1299787,12806,116592,216707,166340,121980,85728,90140,78665,84285,88456,67696,61035,51747,35227,22383
1,2019,합계,남자,611561,5010,41263,106618,92561,68322,48156,52545,46217,44855,38575,25180,18463,12486,7167,4143
2,2019,합계,여자,688226,7796,75329,110089,73779,53658,37572,37595,32448,39430,49881,42516,42572,39261,28060,18240
3,2019,종로구,계,24671,507,3280,3903,2647,1872,1405,1704,1521,1718,1572,1201,1120,1047,704,470
4,2019,종로구,남자,11942,158,1236,2178,1466,959,727,944,880,993,817,562,416,320,170,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2019,송파구,남자,31036,136,978,4623,5697,4352,2954,2958,2301,2119,1834,1176,872,554,296,186
74,2019,송파구,여자,37825,146,1928,6131,5328,4244,2719,2348,1843,2265,2829,2272,2090,1628,1138,916
75,2019,강동구,계,45188,157,1859,5687,5270,4403,3306,3631,3332,3886,4134,3042,2500,1876,1276,829
76,2019,강동구,남자,21674,68,598,2582,2940,2492,1877,2192,2025,2130,1823,1174,833,508,279,153


In [3]:
df1_seoul = df_seoul.iloc[3:77:3,[1,3]]
df1_seoul

,구분,합계
3,종로구,24671
6,중구,22078
9,용산구,35148
12,성동구,41488
15,광진구,58714
18,동대문구,58152
21,중랑구,52196
24,성북구,57832
27,강북구,42791
30,도봉구,32653


In [4]:
import json
geo_path = './data/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [21]:
import folium


map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='OpenStreetMap')

map.choropleth(geo_data=geo_str, data=df1_seoul, columns=['구분', '합계'],
              fill_color='BuPu', key_on='feature.id')

map

In [40]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data=geo_str, data=df1_seoul, columns=['구분', '합계'],
              fill_color='PuRd', key_on='feature.id')

map